<pre>
<img align="center" width="300" src="https://portal.coren-sp.gov.br/wp-content/uploads/2014/11/logo%20uninove.png">
</pre>

<pre>
<img align="left" width="150" src="https://camo.githubusercontent.com/bdc6a3b8963aa99ff57dfd6e1e4b937bd2e752bcb1f1936f90368e5c3a38f670/68747470733a2f2f696d672e736869656c64732e696f2f62616467652f4c6963656e73652d434325323042592d2d5341253230342e302d6c69676874677265792e737667">
</pre>

* __PPGI - UNINOVE__
* Computação Científica com Julia
* Prof. José E. Storópoli
* Aluno: André Santos

# Parte 1 - definição do problema

Exemplo sobre política de empréstimo adaptado de [Jaafar Ballout, 2021](https://www.supplychaindataanalytics.com/linear-programming-in-julia-with-glpk-and-jump/)

| Tipo de empréstimo | Taxas | Índice de inadimplência |
|:------------------ |------:|------------------------:|
| Pessoal            | 0.14  |0.1                      |
| Automóvel          | 0.13  |0.07                     |
| Habitação          | 0.12  |0.03                     |
| Agrícola           | 0.125 |0.05                      |
| Comercial          | 0.10  |0.02                     |

__Problema:__

Um banco está elaborando uma política de empréstimos que envolve um máximo de US$ 12 milhões. A tabela, mostrada acima, fornece os dados sobre os empréstimos disponíveis. É importante notar que as inadimplências são irrecuperáveis e não geram receita de juros. A concorrência com outras instituições financeiras determina a destinação de pelo menos 40\% dos recursos para empréstimos agrícolas e comerciais. Para ajudar o setor imobiliário da região, os empréstimos à habitação devem equivaler a pelo menos 50\% dos empréstimos pessoais e de automóveis. O banco limita o índice geral de inadimplência em todos os empréstimos a no máximo 4\%.

__Variáveis de decisão (empréstimos)__

* x1 = pessoal
* x2 = automóvel
* x3 = habitação
* x4 = agrícola
* x5 = comercial

__Função Objetivo (maximizar o lucro)__

O lucro é a diferença entre as receitas geradas pelas taxas de juros e as perdas devido ao índice de inadimplências:

$lucro = receitas - perdas$ \
$receitas = 0.14((1-0.1)x_1) + 0.13((1-0.07)x_2) + 0.12((1-0.03)x_3) + 0.125((1-0.05)x_4) + 0.10((1-0.02)x_5)$ \
$receitas = 0.14(0.9x_1) + 0.13(0.93x_2) + 0.12(0.97x_3) + 0.125(0.95x_4) + 0.10(0.98x_5)$ \
$receitas = 0.126x_1 + 0.1209x_2 + 0.1164x_3 + 0.11875x_4 + 0.098x_5$ \
$perdas = 0.1x_1 + 0.07x_2 + 0.03x_3 + 0.05x_4 + 0.02x_5$

$ max_z = (0.126x_1 - 0.1x_1) + (0.1209x_2 - 0.07x_2) + (0.1164x_3 - 0.03x_3) + (0.11875x_4 - 0.05x_4) + (0.0098x_5 - 0.02x_5)$ \
$ max_z = 0.26x_1 + 0.0509x_2 + 0.0864x_3 + 0.06875x_4 + 0.0078x_5$

__Restrições__

Soma dos empréstimos deve ser menor ou igual a US$ 12 milhões:

$x_1 + x_2 + x_3 + x_4 + x_5 \leq 12$

Os empréstimos agrícolas e comerciais equivalem a 40\% ou mais de todos os empréstimos:

$x_4 + x_5 \geq 0.4(x_1 + x_2 + x_3 + x_4 + x_5)$ \
$0.4x_1 + 0.4x_2 + 0.4x_3 - 0.6x_4 - 0.6x_5 \leq 0$

Os empréstimos para habitação devem ser maiores ou iguais a 50\% dos empréstimos pessoais e automóveis:

$x_3 \geq 0.5(x_1 + x_2)$ \
$0.5x_1 + 0.5x_2 - x_3 \leq 0$

As inadimplências não devem exceder 4\% de todos os empréstimos:

$0.06x_1 + 0.03x_2 - 0.01x_3 + 0.01x_4 - 0.02x_5 \leq 0$

Não negatividade:

$x_1, x_2, x_3, x_4, x_5 \geq 0$

# Parte 2 - aplicação da programação linear

In [1]:
# Instalar pacotes
using Pkg
Pkg.add("JuMP") # pacote para otimização matemática
Pkg.add("GLPK") # solver

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.7/Project.toml`
  No Changes to `~/.julia/environments/v1.7/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.7/Project.toml`
  No Changes to `~/.julia/environments/v1.7/Manifest.toml`


In [2]:
# Carregar pacotes
using JuMP
using GLPK

In [3]:
# Definir o modelo
BM = Model() # Modelo

# Configurar o otimizador
set_optimizer(BM,GLPK.Optimizer)

# Definir variáveis
@variable(BM, x1>=0)
@variable(BM, x2>=0)
@variable(BM, x3>=0)
@variable(BM, x4>=0)
@variable(BM, x5>=0)

# Definir restrições
@constraint(BM, x1+x2+x3+x4+x5<=12)
@constraint(BM, 0.4x1+0.4x2+0.4x3-0.6x4-0.6x5<=0)
@constraint(BM, 0.5x1+0.5x2-x3<=0)
@constraint(BM, 0.06x1+0.03x2-0.01x3+0.01x4-0.02x5<=0)

# Definir função objetivo
@objective(BM, Max, 0.026x1+0.0509x2+0.0864x3+0.06875x4+0.078x5)

# Executar modelo
optimize!(BM)

In [4]:
# Saída do modelo:
BM

A JuMP Model
Maximization problem with:
Variables: 5
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 4 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 5 constraints
Model mode: AUTOMATIC
CachingOptimizer state: ATTACHED_OPTIMIZER
Solver name: GLPK
Names registered in the model: x1, x2, x3, x4, x5

In [5]:
# Valor da função objetivo (lucro do banco em milhões de dólares):
objective_value(BM)

0.99648

In [6]:
# Sensibilidade
objective_sense(BM)

MAX_SENSE::OptimizationSense = 1

In [7]:
# Variáveis otimizadas:
println("x1 = ", value.(x1),
    "\n","x2 = ",value.(x2),
    "\n", "x3 = ",value.(x3),
    "\n", "x4 = ",value.(x4),
    "\n", "x5 = ",value.(x5))

x1 = 0.0
x2 = 0.0
x3 = 7.199999999999999
x4 = 0.0
x5 = 4.8


__Conclusão:__

O resultado indica que o banco deve gastar, aproximadamente $7,2$ milhões em empréstimos de habitação e $4,8$ milhões de dólares com empréstimos comerciais para obter o maior lucro possível. Assim, o lucro que o banco irá conseguir será de $0,99$ milhões de dólares.

* [Ciência de Dados e Computação Científica com Julia](https://storopoli.io/Computacao-Cientifica/2_BenchmarkTools/)
* [JuMP](https://jump.dev/JuMP.jl/stable/)
* [Jaafar Ballout, 2021](https://www.supplychaindataanalytics.com/linear-programming-in-julia-with-glpk-and-jump/)